In [12]:
import rosbag
import numpy as np
import pandas as pd
import scipy
from scipy import signal
from tf.transformations import unit_vector, vector_norm, quaternion_conjugate, quaternion_multiply, euler_matrix, quaternion_matrix, euler_from_quaternion
# Load methods to extract data from rosbags
from unpack_rosbag import unpack_bag, synchronize_topics

# Standard plotly imports
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = 'plotly_dark'

#### **The following values need to be adjusted for each rosbag**

In [13]:
# Rosbag path
bag_path = '/home/user/rosbags/final/slam/u_s2c2d_hdl.bag'
# bag_path = '/home/user/rosbags/final/slam/u_c2s_half_odom_hdl.bag'
# Load rosbag
bag = rosbag.Bag(bag_path)

# ROS topics
imu_topic = '/zed2i/zed_node/imu/data'
# imu_topic2 = '/imu/data'
odom_topic = '/eGolf/sensors/odometry'

#### Importing the data

In [14]:
# IMU data (angular velocity and linear acceleration)
imu, t_imu = unpack_bag(bag_path, imu_topic, 'imu')
quat, _ = unpack_bag(bag_path, imu_topic, 'quat')
# Odom data (wheel speeds and handwheel angle)
odom, t_odom = unpack_bag(bag_path, odom_topic, 'car_odom')

# Ignore odom if none has been recorded
if len(odom) == 0:
    odom = np.zeros((len(imu), 5))
    is_odom = False
else:
    # Synchronize both topics (IMU is 400 Hz whereas odom is only 100 Hz)
    # Downsamples higher freq signal and matches start and end time
    imu, odom = synchronize_topics(imu, t_imu, odom, t_odom)
    is_odom = True
    
# Split linear acceleration and angular velocity 
ang_vel = imu[:, 0]
lin_acc = imu[:, 1]

# Frequency (of slowest sensor) [Hz]
f = 400
if is_odom:
    f = 100
# Time vector
t = np.linspace(0, odom.shape[0]/f, odom.shape[0])

KeyboardInterrupt: 

Used functions

In [ ]:
from imu_ramp_offline import ImuRampDetect
ird = ImuRampDetect(f)
tf = ird.align_imu(lin_acc)
angles = []
lin_acc_car = []
ang_vel_car = []
for i, _ in enumerate(lin_acc):
    la, av, angs = ird.spin(lin_acc[i], ang_vel[i], odom[i], tf)
    lin_acc_car.append(la)
    ang_vel_car.append(av)
    angles.append(angs)
    
# Convert to numpy array
lin_acc_car = np.asarray(lin_acc_car)
ang_vel_car = np.asarray(ang_vel_car)
# Convert from radians to degree
angles = np.rad2deg(np.asarray(angles))
# Convert to pandas dataframe (for plotting)
df_angles = pd.DataFrame(angles, columns = ["acc", "gyr", "acc_odom", "compl"])

Correct angles by (rpy in deg): 0.70 -22.28 -0.62


AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [ ]:
fig = px.line(df_angles)
fig.update_layout(title="Test", yaxis_title="Tests")

In [ ]:
px.line(angles)

## Transform data from IMU to car frame

In [ ]:
# Ideal normed g measurement in car frame
g_car = (0, 0, 1)

def trafo1(lin_acc):
    """Rotation to align IMU measured g-vector with car z-axis
    :param lin_acc: Linear acceleration while car stands still
    :return:        Quaternion
    """
    g_mag = vector_norm(np.mean(lin_acc, axis=0))
    print('Average linear acceleration magnitude: {}  (should ideally be 9.81 m/s²)'.format(round(g_mag, 2)))
    g_imu = unit_vector(np.mean(lin_acc, axis=0))
    quat = quat_from_vectors(g_imu, g_car)
    return quat

def trafo2(lin_acc, first_rotation):
    """Second rotation to align imu with car frame"""
    # Take average to reduce influence of noise
    lin_acc_avg = np.mean(lin_acc, axis=0)
    # Get rotation matrix from first alignment
    rot_mat1 = quaternion_matrix(first_rotation)[:3, :3]
    # Apply first rotation (trafo1) for z axis alignment
    lin_acc_rot1 = np.inner(rot_mat1, lin_acc_avg)

    # Ignore gravity (because this is already aligned)
    lin_acc_rot1[2] = 0
    # Get quaternion to rotate, such that forward acc is only measured by x-axis
    quat2 = quat_from_vectors(lin_acc_rot1, (1, 0, 0))

    # Apply second rotation to first (always in reverse order) to get final rotation
    quat = quaternion_multiply(quat2, first_rotation)
    # Get euler angles to show the difference between the two frames
    euler_angles = euler_from_quaternion(quat)
    print('Correct angles by (rpy in deg): {:.2f} {:.2f} {:.2f}'.format(
        *[np.rad2deg(x) for x in euler_angles]))

    # Get rotation matrix from quaternion
    rot_mat_imu_car = quaternion_matrix(quat)[:3, :3]
    return rot_mat_imu_car

def quat_from_vectors(vec1, vec2):
    """Gets quaternion to align vector 1 with vector 2"""
    # Make sure both vectors are unit vectors
    v1_uv, v2_uv = unit_vector(vec1), unit_vector(vec2)
    cross_prod = np.cross(v1_uv, v2_uv)
    dot_prod = np.dot(v1_uv, v2_uv)

    # Rotation axis
    axis = cross_prod / vector_norm(cross_prod)
    # Rotation angle (rad)
    ang = np.arccos(dot_prod)

    # Quaternion ([x,y,z,w])
    quat = np.append(axis*np.sin(ang/2), np.cos(ang/2))
    return quat

def car_starts_driving(wheel_speed):
    """Returns index where car starts driving"""
    start = np.nonzero(wheel_speed)[0][0]
    return start

# Check at which moment car starts driving (Rear left wheel speed as input)
#! Added negative buffer (because odom does seem to be a bit behind on some wheels)
if not is_odom:
    # Use first second
    start_idx = f
else:
    start_idx = car_starts_driving(odom[:, 2]) - 50

# Average linear acceleration where car stands still
lin_acc_still = lin_acc[:start_idx]
# Magnitude g (needed for later)
g_mag = vector_norm(np.mean(lin_acc_still, axis=0))
# First transformation (z-axis alignment)
quat1 = trafo1(lin_acc_still)

# Take 1s during first car acceleration
lin_acc_go = lin_acc[start_idx:start_idx + f]
# Second transformation (x- and y-axis alignment)
tf_imu_car = trafo2(lin_acc, quat1)

# Apply transformation to imu data
lin_acc_car = np.inner(tf_imu_car, lin_acc).T
ang_vel_car = np.inner(tf_imu_car, ang_vel).T

In [ ]:
fig = px.line(pd.DataFrame(data=np.hstack([lin_acc, lin_acc_car]), columns=['x0','y0','z0','x1','y1','z1'], index=t))
fig.update_layout(xaxis_title='Time [s]', yaxis_title='$\mathrm{[ms^{-2}]}$', 
                  title='Linear acceleration in IMU frame [0], in car frame [1]')
fig.for_each_trace(lambda trace: trace.update(visible='legendonly')
                  if trace.name in ['x0','y0','z0'] else ())
fig.show()
fig = px.line(pd.DataFrame(data=np.hstack([ang_vel, ang_vel_car]), columns=['x0','y0','z0','x1','y1','z1'], index=t))
fig.update_layout(xaxis_title='Time [s]', yaxis_title='$\mathrm{[ms^{-2}]}$', 
                  title='Angular velocity in IMU frame [0], in car frame [1]')
fig.for_each_trace(lambda trace: trace.update(visible='legendonly')
                  if trace.name in ['x0','y0','z0'] else ())
fig.show()

In [ ]:
# px.line(lin_acc_car[:,1])

from scipy import signal
import matplotlib.pyplot as plt

N = 1       # Filter order
w0 = 2      # Cut-off frequency
fs = 400    # Sample frequency
b, a = signal.butter(N, w0, btype="low", fs=fs)
w, h = signal.freqz(b, a)

plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()

In [ ]:

def butter_lowpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

# Filter requirements.
T = 5.0         # Sample Period
fs = 30.0       # sample rate, Hz
cutoff = 2      # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
nyq = 0.5 * fs  # Nyquist Frequency
order = 2       # sin wave can be approx represented as quadratic
n = int(T * fs) # total number of samples

# Filter the data, and plot both the original and filtered signals.
y = butter_lowpass_filter(data, cutoff, fs, order)

fig = go.Figure()
fig.add_trace(go.Scatter(
            y = data[:4000],
            line =  dict(shape =  'spline' ),
            name = 'signal with noise'
            ))
fig.add_trace(go.Scatter(
            y = y[:4000],
            line =  dict(shape =  'spline' ),
            name = 'filtered signal'
            ))
fig.show()

In [ ]:
data = lin_acc_car[:, 0]

def butter_lowpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

# Filter requirements.
# T = 5.0         # Sample Period
fs = 400.0      # sample rate, Hz
cutoff = 2      # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
nyq = 0.5 * fs  # Nyquist Frequency
order = 2       # sin wave can be approx represented as quadratic
# n = int(T * fs) # total number of samples

# Filter the data, and plot both the original and filtered signals.
y = butter_lowpass_filter(data, cutoff, fs, order)

fig = go.Figure()
fig.add_trace(go.Scatter(
            y = data,
            line =  dict(shape =  'spline' ),
            name = 'signal with noise'
            ))
fig.add_trace(go.Scatter(
            y = y,
            line =  dict(shape =  'spline' ),
            name = 'filtered signal'
            ))
fig.show()

In [ ]:
# Cut-off frequencies to test
f_array = np.linspace(1, 10, 10)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=data))
# Add traces, one for each slider step

for cutoff in f_array:
    fig.add_trace(
        go.Scatter(
            visible=False,
            x=t,
            y=butter_lowpass_filter(data, cutoff, fs, order)))
fig.data[0].visible = True
# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [True] + [False] * len(fig.data)},
              {"title": "Low pass with cut-off: {:.3f} Hz".format(f_array[i-1])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
sliders = [dict(
    active=0,
    currentvalue={"prefix": "alpha: "},
    steps=steps)]
fig.update_layout(sliders=sliders, xaxis_title='Time [s]', yaxis_title='[deg]', title='Complementary Filter compared to IMU+Odom filt with moving avg')
fig.show()

In [ ]:
f0 = 4
a = 1/(2 * np.pi * f0)
a

In [ ]:
1 / (2*np.pi * (1-a))

In [ ]:
np.deg2rad(5)

In [ ]:
tau = 10
alpha = tau / (tau + 1.0/f)
alpha

In [ ]:
def butter_highpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

# Cut-off frequencies to test
f_array = np.linspace(1, 10, 10)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=pitch_from_vel(ang_vel_car[:, 1])))
# Add traces, one for each slider step

for cutoff in f_array:
    fig.add_trace(
        go.Scatter(
            visible=False,
            x=t,
            y=pitch_from_vel(butter_highpass_filter(ang_vel_car[:, 1], 1/cutoff, fs, order))))
fig.data[0].visible = True
# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [True] + [False] * len(fig.data)},
              {"title": "Low pass with cut-off: {:.3f} Hz".format(f_array[i-1])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
sliders = [dict(
    active=0,
    currentvalue={"prefix": "alpha: "},
    steps=steps)]
fig.update_layout(sliders=sliders, xaxis_title='Time [s]', yaxis_title='[deg]', title='Complementary Filter compared to IMU+Odom filt with moving avg')
fig.show()

## Linear acceleration
Calculate pitch angle using only the linear acceleration measurements

In [ ]:
# Only acceleration measured along x-axis is of interest
acc_x_imu = lin_acc_car[:,0]

# Filter signal
# Using moving average (basically low pass)
acc_x_imu_filt = ma_filt(acc_x_imu, 50)
# Using Butterworth filter (only works offline)
fc = 1 # Cutoff frequency in Hz
w = fc / (f/2.0) # Normalize fc (Nyquist)
# num, denom of IIR 4-th order butterworth filter with cutoff of 1 Hz
b, a = signal.butter(4, w, 'low')
acc_x_imu_filt_butt = signal.filtfilt(b,a, acc_x_imu)

# Add all to df (for easier plotting)
acc_x_imu_df = pd.DataFrame(data=np.vstack((acc_x_imu, acc_x_imu_filt, acc_x_imu_filt_butt)).T, index=t, 
                                  columns=['raw', 'moving avg filter', 'butterworth'])

# Calculate pitch angle
def pitch_from_acc(acc_x):
    deg = np.degrees(np.arcsin(acc_x / g_mag))
    return deg
car_ang_imu_acc_df = pitch_from_acc(acc_x_imu_df)
    
# Visualize
fig = px.line(car_ang_imu_acc_df)
fig.update_layout(xaxis_title='Time [s]', yaxis_title='[deg]', title='Pitch angle from IMU fwd acc')
fig.show()

# Check how trustworthy accelerometer data is (g magnitude plot)
# fig = px.line(np.linalg.norm(x)/g_mag for x in lin_acc)
# fig.update_layout(xaxis_title='Time [s]', yaxis_title='[g]', title='g magnitude')

## Angular velocity
Calculate pitch angle using only the angular velocity measurements

In [ ]:
# Only angular velocity measured along y-axis is of interest
vel_y_imu = ang_vel_car[:, 1]

# Filter?

# Calculate pitch angle
def pitch_from_vel(ang_vel_y, f=400):
    if is_odom:
        f = 100
    deg = -np.degrees(np.cumsum(ang_vel_y) / f)
    return deg
ang_y_gyr = pitch_from_vel(vel_y_imu)

def remove_drift(ang_y_gyr):
    drift = np.zeros(len(ang_y_gyr))
    # Only remove drift if end position is also on flat ground
    if np.abs((ang_y_gyr[0] - ang_y_gyr[-1])) < 3:
        drift = np.linspace(ang_y_gyr[0], ang_y_gyr[-1], len(ang_y_gyr))
    ang_y_no_drift = ang_y_gyr - drift
    return ang_y_no_drift, drift

# Remove drift (assumes start and end pitch angle are the same)
ang_y_no_drift, drift = remove_drift(ang_y_gyr)

# Add all to df (for easier plotting)
car_ang_imu_vel_df = pd.DataFrame(data=np.vstack((ang_y_gyr, ang_y_no_drift, drift)).T, index=t, 
                                  columns=['raw', 'w/o drift', 'drift'])
# Visualize
fig = px.line(car_ang_imu_vel_df)
# Plot only raw if drift could not be calculated
if not np.any(car_ang_imu_vel_df["drift"]):
    fig = px.line(car_ang_imu_vel_df["raw"])
fig.update_layout(xaxis_title='Time [s]', yaxis_title='[deg]', title='Pitch angle from IMU y ang vel')
fig.show()

## Odometer
Calculate car acceleration from wheel speed measurements and test gravity method (linear acc of imu + acc from odometer)

In [ ]:
# Calculate car velocity from wheel speeds
def car_vel_from_odom(odom):
    #! THIS IS WRONG (but doesnt matter much)
    wheelbase = 2.631 # m
    alpha = (odom[:,2] - odom[:,3]) / wheelbase
    yaw = alpha * 1.0 / f
    # 3.6 to convert from km/h to m/s
    vel_x_car = ((odom[:,2] + odom[:,3]) / 2) * np.cos(yaw) / 3.6
    return vel_x_car

# Derivate acceleration to get velocity
def acc_from_vel(vel):
    acc = []
    for i in range(len(vel)-1):
        a = (vel[i+1] - vel[i]) / (1.0/f)
        acc.append(a)
    acc.append(a)
    return acc

vel_x_car = car_vel_from_odom(odom)
acc_x_car = acc_from_vel(vel_x_car)
# Simple moving average filter
vel_x_car_filt = ma_filt(vel_x_car, 50)
acc_x_car_filt = acc_from_vel(vel_x_car_filt)

vel_x_car_df = pd.DataFrame(data=np.vstack((vel_x_car, vel_x_car_filt)).T, index=t, columns=['raw', 'moving avg filter'])
acc_x_car_df = pd.DataFrame(data=np.vstack((acc_x_car, acc_x_car_filt)).T, index=t, columns=['raw', 'moving avg filter'])
# fig = px.line(vel_x_car_df)
# fig.update_layout(xaxis_title='Time [s]', yaxis_title='$\mathrm{[ms^{-1}]}$', title='Car velocity')
# fig.show()
# fig = px.line(acc_x_car_df)
# fig.update_layout(xaxis_title='Time [s]', yaxis_title='$\mathrm{[ms^{-2}]}$', title='Car acceleration')
# fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)
# fig.show()

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=vel_x_car_df.index, y=vel_x_car_df["raw"], name="vel raw"))
fig.add_trace(go.Scatter(x=vel_x_car_df.index, y=vel_x_car_df["moving avg filter"], name="vel filt"))
fig.add_trace(go.Scatter(x=acc_x_car_df.index, y=acc_x_car_df["moving avg filter"], name="acceleration", ), secondary_y=True)
fig.update_yaxes(title_text="Velocity", secondary_y=False)
fig.update_yaxes(title_text="Acceleration", secondary_y=True)
fig.update_layout(title='Car velocity and derived acceleration')
fig.show()

# Calculate car pitch angle using imu accelerometer and wheel speed measurements
def gravity_method(acc_x_imu, acc_x_car = None):
    if acc_x_car == None:
        acc_x_car = np.zeros(len(acc_x_imu))
    ang_acc_imu = pitch_from_acc(acc_x_imu - acc_x_car)
    return ang_acc_imu

# Filtering both sensor data with the same filter (to keep time synchronized)
acc_x_imu_filt = ma_filt(acc_x_imu, 50)
vel_x_car_filt = ma_filt(car_vel_from_odom(odom), 50)
acc_x_car_filt = acc_from_vel(vel_x_car_filt)
acc_diff = acc_x_imu_filt - acc_x_car_filt
# acc_diff = gravity_method(acc_x_imu_filt, acc_x_car_filt)

fig = px.line(pd.DataFrame(data=np.vstack([acc_x_imu_filt, acc_x_car_filt, acc_diff]).T, index=t,
                           columns=['Filtered IMU', 'Car acc from filtered vel', 'IMU cleaned from car acc']))
fig.update_layout(xaxis_title='Time [s]', yaxis_title='$\mathrm{[ms^{-2}]}$', title='Comparison of both acceleration measurement methods')
fig.show()

In [ ]:
# TODO: Make method more understandable
#       Limit K to 1 if lin_acc data escalates

def complementary_filter2(ang_vel, lin_acc, dt, K):
    # initial estimations
    theta_gk = 0
    theta_ak = 0
    angle_est = []
    theta_w = 0
    
    for i,v in enumerate(lin_acc):
        # Calculate pitch angle using linear acceleration data
        theta_a = np.arcsin(lin_acc[i, 0] / g_mag)
        # Calculate pitch angle using angular velocity
        theta_w += ang_vel[i, 1] / f
        # Normalize linear acceleration
        lin_acc[i] = lin_acc[i] / np.linalg.norm(lin_acc[i])
    
        angle_k = K * theta_gk +(1 - K) * theta_ak
        theta_gk = theta_gk + ang_vel[i, 1] * dt
        theta_ak = np.arctan2(lin_acc[i, 0], np.sqrt(lin_acc[i, 1]**2 + lin_acc[i, 2]**2))
        angle_est.append(angle_k)
    return -np.rad2deg(angle_est)

px.line(complementary_filter2(ang_vel_car, lin_acc_car, 1.0/f, 0.5))

In [ ]:
def complementary_filter(acc_data, gyr_data, alpha):
    angle = 0
    angle_fused = []
    for i,v in enumerate(gyr_data):
        ang_from_acc = np.arcsin(acc_data[i] / g_mag)
        ang_from_vel = - gyr_data[i] * (1.0 / f)
        angle = (1 - alpha) * (angle + ang_from_vel) + alpha * ang_from_acc
        angle_fused.append(angle)
    return np.rad2deg(angle_fused)

px.line(complementary_filter(acc_x_imu, vel_y_imu, 0.01))

In [ ]:
def complementary_filter_advanced(acc_data, gyr_data, alpha):
    angle = 0
    angle_fused = []
    for i,v in enumerate(gyr_data):
        ang_from_acc = np.arcsin(acc_data[i] / g_mag)
        ang_from_vel = - gyr_data[i] * (1.0 / f)
        angle = (1 - alpha) * (angle + ang_from_vel) + alpha * ang_from_acc
        angle_fused.append(angle)
    return np.rad2deg(angle_fused)

##### Testing different parameter values

In [ ]:
step_vals = np.linspace(0.02, 0, 51)

fig = go.Figure()
# fig.add_trace(go.Scatter(x=t, y=car_ang))
# Add traces, one for each slider step
for step in step_vals:
    fig.add_trace(
        go.Scatter(
            visible=False,
            x=t,
            y=complementary_filter(acc_x_imu, vel_y_imu, step)))
fig.data[0].visible = True
# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [True] + [False] * len(fig.data)},
              {"title": "Angle using complementary filter with alpha: {:.3f}".format(step_vals[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
sliders = [dict(
    active=0,
    currentvalue={"prefix": "alpha: "},
    steps=steps)]
fig.update_layout(sliders=sliders, xaxis_title='Time [s]', yaxis_title='[deg]', title='Complementary Filter compared to IMU+Odom filt with moving avg')
fig.show()

With e.g. $\alpha=0.022$ the signal gets significantly smoother. The amplitude of undesired outliers gets more than halfed. But this comes at the cost of an increased time delay. The filtered signal is about 0.5s seconds behind.  
Because the data from IMU accelerometer + odometry is already filtered and thus has a time delay of 0.25s (win size of 0.5s) the total final signal is delayed by 0.25s + 0.5s = 0.75s.

There has to be a trade-off between the accuarcy of the signal and the introduced time delay.

In [ ]:
car_ang = pitch_car_odom(acc_x_imu_filt, acc_x_car_filt)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=car_ang))
win_len = 25
# Add traces, one for each slider step
for step in np.arange(0, 2, 0.1):
    fig.add_trace(
        go.Scatter(
            visible=False,
            x=t,
            y=fuse_gyr(car_ang, gyr_change_detection(vel_y, win_len, step))))
fig.data[0].visible = True
# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [True] + [False] * len(fig.data)},
              {"title": "Fused signal with window length: {}s and threshold of: {} deg/s".format(win_len/100.0, i/10.0)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Threshold: "},
    steps=steps)]
fig.update_layout(sliders=sliders, xaxis_title='Time [s]', yaxis_title='[deg]')
fig.show()

A window length of 0.25s and a threshold of 0.9 deg/s seem to deliver resonably results. The angle, to be corrected by the gyr data, was calculated using the IMU acc + odom data filtered with a moving average filter with a window length of 50, resulting in a delay of 0.25 s. That's why I chose a window length of 0.25s, in an attempt to synchronice both signals.

In [ ]:
def plotly_slider(min_val, max_val, n_step, function, function_params):
    fig = go.Figure()
    # Add traces, one for each slider step
    for step in np.linspace(min_val, max_val, n_step):
        fig.add_trace(
            go.Scatter(
                visible=False,
                y=function(*function_params),
                x=np.linspace(0, len(y), len(y))))
    fig.data[0].visible = True
    # Create and add slider
    steps = []
    for i in range(len(fig.data)):
        step = dict(
            method="update",
            args=[{"visible": [True] + [False] * len(fig.data)},
                {"title": "Step: {:.3f}".format(i)}],  # layout attribute
        )
        step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
        steps.append(step)
    sliders = [dict(
        active=0,
        currentvalue={"prefix": "alpha: "},
        steps=steps)]
    # fig.update_layout(sliders=sliders, xaxis_title='Time [s]', yaxis_title='[deg]', title='Complementary Filter compared to IMU+Odom filt with moving avg')
    return fig

# Testing different high pass filters for the angular velocity measurements

# fc = 1 # Cutoff frequency in Hz
# w = fc / (f/2.0) # Normalize fc (Nyquist)
# # num, denom of IIR 4-th order butterworth filter with cutoff of 1 Hz
# b, a = signal.butter(4, w, 'low')
# acc_x_imu_filtButter = signal.filtfilt(b,a, acc_x_imu)

# fc = 2.95 # Cutoff frequency in Hz
# w = fc / (f/2.0) # Normalize fc (Nyquist)
# b, a = scipy.signal.butter(N=3, Wn=w, btype='high')
# ang_vel_car_filt = np.zeros(ang_vel_car.shape)
# ang_vel_car_filt[:, 0] = scipy.signal.filtfilt(b, a, ang_vel_car[:, 0])
# # ang_vel_car_filt[:, 1] = signal.filtfilt(b, a, ang_vel_car[:, 1])
# # ang_vel_car_filt[:, 2] = signal.filtfilt(b, a, ang_vel_car[:, 2])

def filter_test(sig, order, f_c, f=400):
    w = f_c / (f/2.0) # Normalize fc (Nyquist)
    b, a = scipy.signal.butter(N=order, Wn=w, btype='high')
    signal = np.zeros(sig.shape)
    signal_filt = scipy.signal.filtfilt(b, a, sig)
    signal_filt = ang_vel_to_ang(signal_filt)
    return signal_filt

def ang_vel_to_ang(ang_vel, f=400):
    return np.rad2deg(np.cumsum(ang_vel) / f)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=np.rad2deg(np.cumsum(ang_vel_car[:, 0])/400)))
# Add traces, one for each slider step
for step in np.linspace(0.001, 0.1, 101):
    fig.add_trace(
        go.Scatter(
            visible=False,
            x=t,
            y=filter_test(ang_vel_car[:, 0], 3, step)))
fig.data[0].visible = True
# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [True] + [False] * len(fig.data)},
              {"title": "Angle using complementary filter with alpha: {:.3f}".format(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
sliders = [dict(
    active=0,
    currentvalue={"prefix": "alpha: "},
    steps=steps)]
fig.update_layout(sliders=sliders, xaxis_title='Time [s]', yaxis_title='[deg]', title='Complementary Filter compared to IMU+Odom filt with moving avg')
fig.show()